#**Chapter 08 이미지를 위한 인공 신경망**

#**08-1 인공 신경망**

# 합성곱

합성곱(convolution): 입력 데이터에 도장을 찍어서 유용한 특성만 드러나게 하는 것으로 비유 가능
- 밀집층에선 모든 입력에 가중치를 곱함
- 합성곱은 입력 데이터 전체에 가중치를 적용하는 것이 아니라 일부에 가중치를 곱함

- 밀집층의 뉴런은 입력 개수만큼 10개의 가중치를 가지고 1개의 출력을 만듬
- 합성곱 층의 뉴런은 3개(하이퍼 파라미터)의 가중치를 가지고 8개의 출력을 만듬

**합성곱 신경망(convolutional neural network, CNN)에서는 완전 연결 신경망(밀집층만 사용하여 만든 신경망)과 달리 뉴런을 필터(filter) 혹은 커널(kernel)이라고 부름**

합성곱에서 입력이 2차원 배열이면 필터(도장)도 2차원이어야 함
- 필터의 커널 크기는(3, 3)으로 가정
 - 왼쪽 위 모서리부터 합성곱 시작
 - 입력의 9개 원소와 커널의 9개 가중치를 곱한 후 절편을 더해 1개의 출력을 만듬
 - 이후 필터가 오른쪽으로 한 칸 이동하여 합성곱 수행
 - 더 이상 오른쪽으로 이동할 수 없으므로 왼쪽아래 > 오른쪽으로 이동
 - (4,4)의 입력에 (3,3)의 필터가 왼쪽 위부터 오른쪽 아래로 이동하며 4개의 출력을 만듬
 - **출력을 위치에 맞춰 (2,2) 2차원으로 표현, 이를 특성 맵(feature map)이라고 부**

합성곱 층에서도 여러 개의 필터 사용
- 각기 다른 가중치를 가진 3개의 필터를 사용하면 (2,2) 크기의 특성맵이 차곡차곡 쌓여서(2,2,3) 크기의 3차원 배열이 됨

#케라스 합성곱 층
케라스의 층은 모두 keras.layer 패키지 아래 클래스로 구현
- 특별히 입력 위를 (왼쪽>오른쪽, 위>아래) 이동하는 합성곱은 Conv2D 클래스로 제공

Conv2D 클래스의 첫 번째 매개변수는 필도(도장)의 개수
- kernel_size 매개변수는 필터에 사용할 커널의 크기 지정 (필터의 개수와 커널 크기는 반드시 지정해야 하는 매개변수)

마지막으로 밀집층에서처럼 활성화 함수 지정
- 일반적으로 특성 맵은 활성화 함수를 통과한 값

In [2]:
from tensorflow import keras
keras.layers.Conv2D(10, kernel_size = (3, 3), activation='relu')

합성곱 신경망 : 일반적으로 1개 이상의 합성곱 층을 쓴 인공 신경망

앞서, (4,4) 크기의 입력에 (3,3)크기의 커널을 적요하여 (2,2) 크기의 특성 맵을 만듬
>커널 크기는 (3,3) 그대로 두고 출력의 크기를 (4,4)로 늘리려면?
- 더 큰 입력에 합성곱하는 척이 필요함
 - 실제 입력 크기는 (4,4)이지만 (6,6)크기의 입력에 합성곱을 하는 것처럼 (빈공간 사용)

입력 배열의 주위를 가상의 원소로 채우는 것을 패딩(padding)이라고 함
- 실제 입력 값이 아니기 때문에 패딩은 0으로 채움
 - (4,4) 크기의 입력에 0을 1개 패딩하면 (6,6) 크기의 입력이 됨

**입력과 특성 맵의 크기를 동일하게 만들기 위해 입력 주위에 0으로 패딩하는 것을 세임 패딩(same paddign)이라고 부름**
- 합성곱 신경망에서는 세임 패딩이 많이 사용됨

**패딩 없이 순수한 입력 배열에서만 합성곱을 하여 특성맵을 만드는 경우를 밸리드 패딩(valid padding)이라고 부름**
- 밸리드 패딩은 특성 맵의 크기가 줄어들 수 밖에 없음

패딩을 사용하는 이유?
- 패딩이 없으면, 네 모서리에 위치한 값들은 1번만 계산되는 반면, 다른 값들은 두 번 이상 계산됨
 - 패딩 사용 x : 모서리와 중앙부 픽셀의 참여 비율 = 1:4
 - 1 픽셀 패딩시 : 9:4
 - 2 픽셀 패딩시 : 1:1
- 적절한 패딩은 이미지 주변에 있는 정보를 잃어버리지 않도록 도와줌

케라스 Conv2D 클래스에서는 padding 매개변수로 패딩 지정 가능
- 기본값은 'valid'로 밸리드 패딩
- 세임 패딩을 사용하려면 'same'으로 지정

In [4]:
keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu', padding='same')

**스트라이드(stride): 합성곱 연산시 좌우, 위아래로 이동하는 칸 수**
- 기본 값은 1로 한칸씩 이동
- 케라스 Conv2D의 strides 매개변수의 기본값

In [5]:
keras.layers.Conv2D(10, kernel_size=(3,3), activation='relu', padding='same', strides=1)

strides 매개변수는 오른쪽으로 이동하는 크기와 아래쪽으로 이동하는 크기를 (1,1)과 같이 튜플을 사용해 각각 지정가능
- 그러나 다르게 지정하는 경우는 거의 없음
- 1보다 큰 스트라이드를 사용하는 경우도 거의 없고 기본값 사용

**폴링(pooling)은 합성곱 층에서 만든 특성 맵의 가로세로 크기를 줄이는 역할 수행**
- 그러나 특성 맵의 개수를 줄이진 않음
- 예를 들어 (2,2,3) 크기의 특성맵에 폴링을 적용하면 너비와 높이만 줄어들어 (1,1,3) 크기의 특성 맵이 됨

폴링도 합성곱처럼 입력 위를 지나가면서 도장을 찍지만, 가중치가 없음
- **도장을 찍은 영역에서 가장 큰 값을 고르면 최대 폴링(max pooling)**
- **평균값을 계산하면 평균 폴링(average pooing)이라고 부름**
- 폴링은 합성곱 층과 뚜렷이 구분되기 때문에 폴링 층이라고 부름

폴링은 겹치는 영역이 없음
- 폴링의 크기가 (2,2)인 경우 스트라이드도 2, (3,3)인 경우 스트라이드도 3임
- 케라스에서 MaxPooling2D 클래스로 폴링 수행 가능

In [8]:
keras.layers.MaxPooling2D(2)

MaxPooling2D의 첫 번째 매개변수로 폴링의 크기 지정
- 대부분 폴링의 크기는 2로 가로세로 크기를 절반으로 줄임
- 가로세로 방향의 폴링 크기를 다르게 하려면 (2,3)과 같이 튜플로 지정할 수 있지만 극히 드뭄
- strides와 padding 매개변수 제공, strides의 기본값은 자동으로 폴링의 크기이므로 따로 지정할 필요 없음
- padding의 기본값은 'valid'로 따로 패딩을 하지 않아 이 매개변수를 바꾸는 경우는 거의 없음

In [9]:
# 앞의 코드와 동일
keras.layers.MaxPooling2D(2, strides=2, padding='valid')

평균 폴링을 제공하는 클래스는 AveragePooling2D
- 최대값대신 평균을 계산하는것만 빼면 MaxPooling2D와 매개변수까지 동일
- 대부분 최대 폴링을 많이 사용
 - 평균 폴링은 특성 맵에 있는 중요한 정보를 (평균하여) 희석시킬 수 있기 때문

# 합성곱 신경망의 전체 구조

> 입력 층
- 합성곱 층에서 사용할 커널의 크기는 (3,3) 크기이고 세임 패딩이므로 1픽셀이 입력 데이터 주변에 추가됨
- 패딩은 텐서플로에서 자동으로 추가하므로 수동으로 입력에 작업을 추가할 필요가 없음

> 합성곱 층
- 합성곱의 필터는 3개로 각각(3,3) 크기 가중치를 가지고 있으며 절편이 하나씩 존재함 (가중치는 각기 서로 다름)
- 합성곱의 스트라이드는 따로 언급이 없다면 항상 1
- 만들어지는 특성맵은 입력과 동일한 (4,4)
- 3개의 필터가 하나씩 합성곱의 출력을 만들고 이 출력이 합쳐져서 (4,4,3) 크기의 특성 맵이 만들어짐
- 합성곱 층은 활성화 함수로 렐루함수를 많이 사용함

> 폴링 층
- 보통 (2,2) 폴링을 사용해 특성 맵의 가로 세로 크기를 절반으로 줄임
- 특성 맵의 개수는 변하지 않으므로 (4,4,3)에서 (2,2,3)으로 특성 맵 개수는 유지됨
- 폴링을 사용하는 이유는 합성곱에서 스트라이드를 크게하여 특성 맵을 줄이는 것 보다 폴링 층에서 크기를 줄이는 것이 경험적으로 더 나은 성능을 내기 때문
- 밀집층인 출력층에 전달하려면 3차원 배열을 1차원으로 펼쳐야 함
 - Flatten 클래스, 12개의 원소를 가진 1차원 배열이고 출력층의 입력이됨

> 밀집층
- 출력층에는 3개의 뉴런, 즉 3개의 클래스를 분류하는 다중 분류 문제
- 출력층에서 계산된 값은 소프트맥스 활서와 함수를 거쳐 최종 예측 확률이 됨


컬러 이미지를 사용한 합성곱
- 컬러 이미지는 rgb 채널로 구성되어 있기 때문에 컴퓨터는 이를 3차원 배열로 표시함
- 따라서 필터의 커널 크기도 (3, 3)이 아니라 (3,3,3)이 되어야 함
- 마지막 3이 깊이 차원으로 배열의 깊이는 항상 입력의 깊이와 같음
- (3,3,3) 영역에 해당하는 27개의 원소에 27개의 가중치를 곱하고 절편을 더하여 계산
- 중요한 점은 입력이나 필터의 차원이 몇 개인지 상관없이 항상 출력은 하나의 값, 즉 특성 맵에 있는 한 원소가 채워짐
- 케라스의 합성곱 층은 항상 3차원 입력을 기대하여 흑백 이미지인 경우 깊이 차원이 1인 3차원 배열로 변환하여 전달

합성곱 층 - 풀링 층 다음 다시 합성곱 층이 오는 경우
> 첫번 째 합성곱 층의 필터 개수가 5개
- 첫 번째 풀링 층을 통과한 특성 맵의 크기가 (4,4,5)

> 두 번째 합성곱 층에서 필터의 너비와 높이가 각각 3이라면 이 필터의 커널 크기는 (3,3,5)
- 입력의 깊이와 필터의 깊이는 같아야 함으로
- 45개의 가중치를 곱하고 절편을 더한 합성곱의 결과는 1개의 출력을 만듬

> 두번째 합성곱 층의 필터 개수가 10개라면 만들어진 특성 맵의 크기는 (2,2,10)

**이렇게 합성곱 신경망은 너비와 높이는 점점 줄어들고 깊이는 점점 깊어지는 것이 특징**

합성곱 신경망에서 필터는 이미지에 있는 어떤 특징을 찾는다고 생각할 수 있음
- 처음엔 간단한 기본적인 특징 (직선, 곡선 등)을 찾고 층이 깊어질수록 다양하고 구체적인 특징을 감지할 수 있도록 필터의 개수를 늘림
- 어떤 이미지의 어느 위치에 놓이더라도 쉽게 감지할 수 있도록 너비와 높이 차원은 점점 줄여감